코드 출처: https://github.com/deepseasw/seq2seq_chatbot<br>

In [1]:
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, losses, metrics
from tensorflow.keras import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os
import re

from konlpy.tag import Okt

<br>
<br>

# 데이터 로드

Seq2Seq에서의 임베딩이 이전 예제와 다른 점은 아래와 같이 태그를 사용한다는 것입니다.<br>
임베딩의 0~3번째에 각각 PADDING, START, END, OOV 태그를 넣습니다.<br>
사실 그냥 똑같은 단어라고 보시면 됩니다. 다만 이 단어들이 Seq2Seq의 동작을 제어합니다. <br>
<br>
예를 들어, 디코더 입력에 START가 들어가면 디코딩의 시작을 의미합니다. 반대로 디코더 출력에 END가 나오면 디코딩을 종료합니다.<br>

In [3]:
# 태그 단어
PAD = "<PADDING>"   # 패딩
STA = "<START>"     # 시작
END = "<END>"       # 끝
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# 데이터 타입
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수
max_sequences = 30

# 임베딩 벡터 차원
embedding_dim = 100

# LSTM 히든레이어 차원
lstm_hidden_dim = 128

# 정규 표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

# 챗봇 데이터 로드
chatbot_data = pd.read_csv('./ChatbotData.csv', encoding='utf-8')
question, answer = list(chatbot_data['Q']), list(chatbot_data['A'])

챗봇의 훈련을 위해서 송영숙님이 공개한 한글 데이터셋을 로드합니다.<br>
질문과 대답, 감정 등 총 3개의 항목으로 구성되어 있습니다.<br>
감정 분류는 Seq2Seq에 필요가 없기 때문에 사용하지 않습니다.<br>
https://github.com/songys/Chatbot_data<br>


In [4]:
# 데이터 개수
len(question)


11824

In [5]:
# 데이터의 일부만 학습에 사용
question = question[:100]
answer = answer[:100]

# 챗봇 데이터 출력
for i in range(10):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()


Q : 12시 땡!
A : 하루가 또 가네요.

Q : 1지망 학교 떨어졌어
A : 위로해 드립니다.

Q : 3박4일 놀러가고 싶다
A : 여행은 언제나 좋죠.

Q : 3박4일 정도 놀러가고 싶다
A : 여행은 언제나 좋죠.

Q : PPL 심하네
A : 눈살이 찌푸려지죠.

Q : SD카드 망가졌어
A : 다시 새로 사는 게 마음 편해요.

Q : SD카드 안돼
A : 다시 새로 사는 게 마음 편해요.

Q : SNS 맞팔 왜 안하지ㅠㅠ
A : 잘 모르고 있을 수도 있어요.

Q : SNS 시간낭비인 거 아는데 매일 하는 중
A : 시간을 정하고 해보세요.

Q : SNS 시간낭비인데 자꾸 보게됨
A : 시간을 정하고 해보세요.



<br>
<br>

# 단어 사전 생성

In [6]:
# 형태소분석 함수
def pos_tag(sentences):
    
    # KoNLPy 형태소분석기 설정
    tagger = Okt()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    
    # 모든 문장 반복
    for sentence in sentences:
        # 특수기호 제거
        # RE_FILTER에 해당되는 정규표현식 char에 대하여 "" ()로 바꾸어라
        sentence = re.sub(RE_FILTER, "", sentence)
        
        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
        
    return sentences_pos

In [7]:
# 형태소분석 수행
question = pos_tag(question)
answer = pos_tag(answer)

# 형태소분석으로 변환된 챗봇 데이터 출력
for i in range(10):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()

Q : 12시 땡
A : 하루 가 또 가네요

Q : 1 지망 학교 떨어졌어
A : 위로 해 드립니다

Q : 3 박 4일 놀러 가고 싶다
A : 여행 은 언제나 좋죠

Q : 3 박 4일 정도 놀러 가고 싶다
A : 여행 은 언제나 좋죠

Q : PPL 심하네
A : 눈살 이 찌푸려지죠

Q : SD 카드 망가졌어
A : 다시 새로 사는 게 마음 편해요

Q : SD 카드 안 돼
A : 다시 새로 사는 게 마음 편해요

Q : SNS 맞팔 왜 안 하지 ㅠㅠ
A : 잘 모르고 있을 수도 있어요

Q : SNS 시간 낭비 인 거 아는데 매일 하는 중
A : 시간 을 정 하고 해보세요

Q : SNS 시간 낭비 인데 자꾸 보게 됨
A : 시간 을 정 하고 해보세요



In [8]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)
sentences.extend(answer)

words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]

질문과 대답 문장들을 합쳐서 전체 단어 사전을 만듭니다.<br>
자연어처리에서는 항상 이렇게 단어를 인덱스에 따라 정리를 해야 합니다.<br>
<br>
그래야지 문장을 인덱스 배열로 바꿔서 임베딩 레이어에 넣을 수 있습니다.<br>
또한 모델의 출력에서 나온 인덱스를 다시 단어로 변환하는데도 필요합니다.<br>
<br>

In [9]:
# 단어 개수
len(words)

454

In [10]:
# 단어 출력
words

['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '맞팔',
 '을',
 '는',
 '보게',
 '되나',
 '싶은데',
 '하세요',
 '에',
 '나오세요',
 '왔나',
 '그런거니',
 '처럼',
 '들',
 '필요하죠',
 '갑작스러웠나',
 '박',
 '돌아가서',
 '선생님',
 '물어',
 '책임질',
 '벗어나는',
 'PPL',
 '죠',
 '난다',
 '제일',
 '사랑',
 '강원도',
 '가까워질',
 '다시',
 '옷',
 '보고',
 '정',
 '씨방',
 '알',
 '나',
 '많이',
 '맛있게',
 '사세요',
 '언제나',
 '있는',
 '어디',
 '부모님',
 '빼고',
 '먹었는데',
 '이야기',
 '시간',
 '으로',
 '휴식',
 '짧죠',
 '될',
 '알려',
 '있을',
 '학교',
 '무시',
 '도',
 '보내고',
 '먹을까',
 '믿어',
 '개시',
 '켜고',
 '거',
 '이랑',
 '득템',
 '바라요',
 '했길',
 '그',
 '습관',
 '싶다',
 '시켜',
 '기회',
 '준',
 '돼겠지',
 '붙잡고',
 '개인',
 '의',
 '좋아요',
 '설움',
 '카드',
 '못',
 '입어볼까',
 '개강',
 '해보여',
 '막',
 '간접흡연',
 '해볼까',
 '이나',
 'ㅠㅠ',
 '망가졌어',
 '와',
 '좋겠네요',
 '3',
 '있을까',
 '살찐',
 '쇼핑',
 '쫄딱',
 '떨어졌어',
 '가야',
 '없죠',
 '간다',
 '개념',
 '가보세요',
 '좋아해주세요',
 '어서',
 '것',
 '켜놓고',
 '데',
 '새',
 '서먹해졌어',
 '간장',
 '때',
 '먹고',
 '여행',
 '했잖아',
 '지망',
 '있어요',
 '있으면',
 '하고',
 '생활',
 '하자고',
 '사는',
 '가서',
 '수도',
 '드는',
 '키워',
 '끼리',
 '비싼데',
 '걸린',
 '땀',
 '좋겠다',
 '버렸어',
 '자꾸

In [11]:
# 단어와 인덱스의 딕셔너리 생성, Series.count_values()도 가능
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [12]:
# 단어 -> 인덱스
# 문장을 인덱스로 변환하여 모델 입력으로 사용
word_to_index

{'<PADDING>': 0,
 '<START>': 1,
 '<END>': 2,
 '<OOV>': 3,
 '맞팔': 4,
 '을': 5,
 '는': 6,
 '보게': 7,
 '되나': 8,
 '싶은데': 9,
 '하세요': 10,
 '에': 11,
 '나오세요': 12,
 '왔나': 13,
 '그런거니': 14,
 '처럼': 15,
 '들': 16,
 '필요하죠': 17,
 '갑작스러웠나': 18,
 '박': 19,
 '돌아가서': 20,
 '선생님': 21,
 '물어': 22,
 '책임질': 23,
 '벗어나는': 24,
 'PPL': 25,
 '죠': 26,
 '난다': 27,
 '제일': 28,
 '사랑': 29,
 '강원도': 30,
 '가까워질': 31,
 '다시': 32,
 '옷': 33,
 '보고': 34,
 '정': 35,
 '씨방': 36,
 '알': 37,
 '나': 38,
 '많이': 39,
 '맛있게': 40,
 '사세요': 41,
 '언제나': 42,
 '있는': 43,
 '어디': 44,
 '부모님': 45,
 '빼고': 46,
 '먹었는데': 47,
 '이야기': 48,
 '시간': 49,
 '으로': 50,
 '휴식': 51,
 '짧죠': 52,
 '될': 53,
 '알려': 54,
 '있을': 55,
 '학교': 56,
 '무시': 57,
 '도': 58,
 '보내고': 59,
 '먹을까': 60,
 '믿어': 61,
 '개시': 62,
 '켜고': 63,
 '거': 64,
 '이랑': 65,
 '득템': 66,
 '바라요': 67,
 '했길': 68,
 '그': 69,
 '습관': 70,
 '싶다': 71,
 '시켜': 72,
 '기회': 73,
 '준': 74,
 '돼겠지': 75,
 '붙잡고': 76,
 '개인': 77,
 '의': 78,
 '좋아요': 79,
 '설움': 80,
 '카드': 81,
 '못': 82,
 '입어볼까': 83,
 '개강': 84,
 '해보여': 85,
 '막': 86,
 '간접흡연': 87,
 '

In [13]:
# 인덱스 -> 단어
# 모델의 예측 결과인 인덱스를 문장으로 변환시 사용
index_to_word


{0: '<PADDING>',
 1: '<START>',
 2: '<END>',
 3: '<OOV>',
 4: '맞팔',
 5: '을',
 6: '는',
 7: '보게',
 8: '되나',
 9: '싶은데',
 10: '하세요',
 11: '에',
 12: '나오세요',
 13: '왔나',
 14: '그런거니',
 15: '처럼',
 16: '들',
 17: '필요하죠',
 18: '갑작스러웠나',
 19: '박',
 20: '돌아가서',
 21: '선생님',
 22: '물어',
 23: '책임질',
 24: '벗어나는',
 25: 'PPL',
 26: '죠',
 27: '난다',
 28: '제일',
 29: '사랑',
 30: '강원도',
 31: '가까워질',
 32: '다시',
 33: '옷',
 34: '보고',
 35: '정',
 36: '씨방',
 37: '알',
 38: '나',
 39: '많이',
 40: '맛있게',
 41: '사세요',
 42: '언제나',
 43: '있는',
 44: '어디',
 45: '부모님',
 46: '빼고',
 47: '먹었는데',
 48: '이야기',
 49: '시간',
 50: '으로',
 51: '휴식',
 52: '짧죠',
 53: '될',
 54: '알려',
 55: '있을',
 56: '학교',
 57: '무시',
 58: '도',
 59: '보내고',
 60: '먹을까',
 61: '믿어',
 62: '개시',
 63: '켜고',
 64: '거',
 65: '이랑',
 66: '득템',
 67: '바라요',
 68: '했길',
 69: '그',
 70: '습관',
 71: '싶다',
 72: '시켜',
 73: '기회',
 74: '준',
 75: '돼겠지',
 76: '붙잡고',
 77: '개인',
 78: '의',
 79: '좋아요',
 80: '설움',
 81: '카드',
 82: '못',
 83: '입어볼까',
 84: '개강',
 85: '해보여',
 86: '막',
 87: '간접흡연',
 8

<br>
<br>

# 전처리

In [14]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)

Seq2Seq에서는 학습시 다음과 같이 총 3개의 데이터가 필요합니다.

인코더 입력 : 12시 땡<br>
디코더 입력 : START 하루 가 또 가네요<br>
디코더 출력 : 하루 가 또 가네요 END
<br>
원래 Seq2Seq는 디코더의 현재 출력이 디코더의 다음 입력으로 들어갑니다.<br>
다만 학습에서는 굳이 이렇게 하지 않고 디코더 입력과 디코더 출력의 데이터를 각각 만듭니다. <br>
<br>
그러나 예측시에는 이런 방식이 불가능합니다.<br>
출력값을 미리 알지 못하기 때문에, 디코더 입력을 사전에 생성할 수가 없습니다.<br>
이런 문제를 해결하기 위해 훈련 모델과 예측 모델을 따로 구성해야 합니다.<br>
모델 생성 부분에서 다시 자세히 설명을 드리겠습니다.<br>
<br>
<br>

In [15]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)

print('첫번째 question:', question[0])
# 첫 번째 인코더 입력 출력 (12시 땡)
print('첫번째 Encoder 입력:',x_encoder[0])
print()

# 출력된 125, 308은 word의 index
print('index_to_word[ 125 ]:', index_to_word[ 125 ])
print('index_to_word[ 308 ]:', index_to_word[ 308 ])

첫번째 question: 12시 땡
첫번째 Encoder 입력: [292 139   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]

index_to_word[ 125 ]: 수도
index_to_word[ 308 ]: 너무


In [16]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

print('첫번째 answer:', answer[0])
print('첫번째 Decoder input:', x_decoder[0])
print()

# 출력된 1, 259, 223, 114, 74는 word의 index
print('index_to_word[   1 ]:', index_to_word[   1 ])
print('index_to_word[ 259 ]:', index_to_word[ 259 ])
print('index_to_word[ 223 ]:', index_to_word[ 223 ])
print('index_to_word[ 114 ]:', index_to_word[ 114 ])
print('index_to_word[  74 ]:', index_to_word[  74 ])

첫번째 answer: 하루 가 또 가네요
첫번째 Decoder input: [  1 232 336 437 321   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]

index_to_word[   1 ]: <START>
index_to_word[ 259 ]: 무모한
index_to_word[ 223 ]: 적
index_to_word[ 114 ]: 먹고
index_to_word[  74 ]: 준


In [17]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)

# 첫 번째 디코더 목표 출력 (하루 가 또 가네요 END)
y_decoder[0]

print('첫번째 answer:', answer[0])
print('첫번쨰 Decoder onput:', y_decoder[0])
print()

# 출력된 1, 259, 223, 114, 74는 word의 index
print('index_to_word[ 259 ]:', index_to_word[ 259 ])
print('index_to_word[ 223 ]:', index_to_word[ 223 ])
print('index_to_word[ 114 ]:', index_to_word[ 114 ])
print('index_to_word[  74 ]:', index_to_word[  74 ])
print('index_to_word[   2 ]:', index_to_word[   2 ])

첫번째 answer: 하루 가 또 가네요
첫번쨰 Decoder onput: [232 336 437 321   2   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]

index_to_word[ 259 ]: 무모한
index_to_word[ 223 ]: 적
index_to_word[ 114 ]: 먹고
index_to_word[  74 ]: 준
index_to_word[   2 ]: <END>


In [18]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)))
one_hot_data[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
#one_hot_data의 첫 row에 259번째 column에 '하루'
#one_hot_data의 첫 row에 223번째 column에 '가'
#one_hot_data의 첫 row에 114번째 column에 '또'.... 를 1로 변경
y_decoder[0]

array([232, 336, 437, 321,   2,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [20]:
# 디코더 목표를 원핫인코딩으로 변환
# 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
for i, sequence in enumerate(y_decoder):
    for j, index in enumerate(sequence):
        one_hot_data[i, j, index] = 1

# 디코더 목표 설정
y_decoder = one_hot_data

# 첫 번째 디코더 목표 출력
y_decoder[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [21]:
# 259번째 index (하루) 가 1임
y_decoder[0][0][112]

0.0

인코더 입력과 디코더 입력은 임베딩 레이어에 들어가는 인덱스 배열입니다.<br>
반면에 디코더 출력은 원핫인코딩 형식이 되어야 합니다.<br>
디코더의 마지막 Dense 레이어에서 softmax로 나오기 때문입니다.<br>

<br>
<br>

# 모델 생성

In [22]:
#--------------------------------------------
# 훈련 모델 인코더 정의
#--------------------------------------------

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim,
                                                dropout=0.1,
                                                recurrent_dropout=0.5,
                                                return_state=True)(encoder_outputs)

# 히든 상태와 셀 상태를 하나로 묶음
# Decoder의 initial state에 넣어주기 위함
# 즉, input sentence의 모든 정보를 통해 Decoding 하기 위함
encoder_states = [state_h, state_c]


In [23]:
encoder_outputs

<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'lstm')>

In [24]:
encoder_states

[<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'lstm')>,
 <KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'lstm')>]

In [25]:
#--------------------------------------------
# 훈련 모델 디코더 정의
#--------------------------------------------

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(lstm_hidden_dim,
                           dropout=0.1,
                           recurrent_dropout=0.5,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


지금까지의 예제는 Sequential 방식의 모델이었습니다.<br>
하지만 이번에는 함수형 API 모델을 사용했습니다.<br>
인코더와 디코더가 따로 분리되어야 하는데, 단순히 레이어를 추가하여 붙이는 순차형으로는 구현이 불가능하기 때문입니다. <br>
<br>


In [26]:
#--------------------------------------------
# 훈련 모델 정의
#--------------------------------------------

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])    

Model() 함수로 입력과 출력을 따로 설정하여 모델을 만듭니다.<br>
그다음 compile과 fit은 이전과 동일하게 적용하시면 됩니다.<br>
<br>
<br>

In [27]:
#--------------------------------------------
#  예측 모델 인코더 정의
#--------------------------------------------

# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = models.Model(encoder_inputs, encoder_states)

#--------------------------------------------
# 예측 모델 디코더 정의
#--------------------------------------------

# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

예측 모델은 이미 학습된 훈련 모델의 레이어들을 그대로 재사용합니다. 예측 모델 인코더는 훈련 모델 인코더과 동일합니다. 그러나 예측 모델 디코더는 매번 LSTM 상태값을 입력으로 받습니다. 또한 디코더의 LSTM 상태를 출력값과 같이 내보내서, 다음 번 입력에 넣습니다. 

이렇게 하는 이유는 LSTM을 딱 한번의 타임 스텝만 실행하기 때문입니다. 그래서 매번 상태값을 새로 초기화 해야 합니다. 이와 반대로 훈련할때는 문장 전체를 계속 LSTM으로 돌리기 때문에 자동으로 상태값이 전달됩니다. 

In [28]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 100)    45400       ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 100)    45400       ['input_2[0][0]']                
                                                                                              

In [29]:
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         45400     
                                                                 
 lstm (LSTM)                 [(None, 128),             117248    
                              (None, 128),                       
                              (None, 128)]                       
                                                                 
Total params: 162,648
Trainable params: 162,648
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Decoder_model에는 
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, None, 100)    45400       ['input_2[0][0]']                
                                                                                                  
 input_3 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 128)]        0           []                               
                                                                                            

<br>
<br>

# 훈련 및 테스트

In [31]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        elif vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence += vocabulary[OOV_INDEX]
            
        # 빈칸 추가
        sentence += ' '

    return sentence

In [32]:
# 에폭 반복
for epoch in range(20):
    print('Total Epoch :', epoch + 1)

    # 훈련 시작
    history = model.fit([x_encoder, x_decoder],
                        y_decoder,
                        epochs=100,
                        batch_size=64,
                        verbose=0)
    
    # 정확도와 손실 출력
    print('accuracy :', history.history['acc'][-1])
    print('loss :', history.history['loss'][-1])
    
    # 문장 예측 테스트
    # (3 박 4일 놀러 가고 싶다) -> (여행 은 언제나 좋죠)
    input_encoder = x_encoder[2].reshape(1, x_encoder[2].shape[0])
    input_decoder = x_decoder[2].reshape(1, x_decoder[2].shape[0])
    results = model.predict([input_encoder, input_decoder])
    
    # 결과의 원핫인코딩 형식을 인덱스로 변환
    # 1축을 기준으로 가장 높은 값의 위치를 구함
    indexs = np.argmax(results[0], 1) 
    
    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
    print(sentence)
    print()


Total Epoch : 1
accuracy : 0.8416666388511658
loss : 0.7041441202163696
1/1 [==============================] - 1s 963ms/step
저 도 이 

Total Epoch : 2
accuracy : 0.9253333210945129
loss : 0.3601609468460083
1/1 [==============================] - 0s 50ms/step
맛있게 도 언제나 좋죠 

Total Epoch : 3
accuracy : 0.9679999947547913
loss : 0.1488395631313324
1/1 [==============================] - 0s 31ms/step
빨리 은 언제나 좋죠 

Total Epoch : 4
accuracy : 0.9743333458900452
loss : 0.08869320154190063
1/1 [==============================] - 0s 63ms/step
여행 은 언제나 좋죠 

Total Epoch : 5
accuracy : 0.9776666760444641
loss : 0.06836984306573868
1/1 [==============================] - 0s 29ms/step
여행 은 언제나 좋죠 

Total Epoch : 6
accuracy : 0.9816666841506958
loss : 0.05431702733039856
1/1 [==============================] - 0s 48ms/step
여행 은 언제나 좋죠 

Total Epoch : 7
accuracy : 0.9883333444595337
loss : 0.04235469177365303
1/1 [==============================] - 0s 40ms/step
여행 은 언제나 좋죠 

Total Epoch : 8
accuracy : 0.99266

학습이 진행될수록 예측 문장이 제대로 생성되는 것을 볼 수 있습니다. 다만 여기서의 예측은 단순히 테스트를 위한 것이라, 인코더 입력과 디코더 입력 데이터가 동시에 사용됩니다. 아래 문장 생성에서는 예측 모델을 적용하기 때문에, 오직 인코더 입력 데이터만 집어 넣습니다.
<br>
<br>
<br>

In [33]:
# 모델 저장
encoder_model.save('./model/seq2seq_chatbot_encoder_model.h5')
decoder_model.save('./model/seq2seq_chatbot_decoder_model.h5')

# 인덱스 저장
with open('./model/word_to_index.pkl', 'wb') as f:
    pickle.dump(word_to_index, f, pickle.HIGHEST_PROTOCOL)
with open('./model/index_to_word.pkl', 'wb') as f:
    pickle.dump(index_to_word, f, pickle.HIGHEST_PROTOCOL)    

<br>
<br>

# 문장 생성

In [34]:
# 모델 파일 로드
encoder_model = models.load_model('./model/seq2seq_chatbot_encoder_model.h5')
decoder_model = models.load_model('./model/seq2seq_chatbot_decoder_model.h5')

# 인덱스 파일 로드
with open('./model/word_to_index.pkl', 'rb') as f:
    word_to_index = pickle.load(f)
with open('./model/index_to_word.pkl', 'rb') as f:
    index_to_word = pickle.load(f)    

In [35]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [36]:
# 텍스트 생성
def generate_text(input_seq):
    
    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)

    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))
    
    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = STA_INDEX
    
    # 인덱스 초기화
    indexs = []
    
    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict(
                                                [target_seq] + states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)
        
        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break

        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index
        
        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
        
    return sentence

제일 첫 단어는 START로 시작합니다. 그리고 출력으로 나온 인덱스를 디코더 입력으로 넣고 다시 예측을 반복합니다. 상태값을 받아 다시 입력으로 같이 넣는 것에 주의하시기 바랍니다. END 태그가 나오면 문장 생성을 종료합니다.
<br>
<br>
<br>

In [37]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('휴강이 좋아요')
input_seq


array([[  3, 297,  79,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [38]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


1/1 [==============================] - 0s 40ms/step


'그 사람 도 그럴 거 예요 '

데이터셋에 있는 문장과 똑같은 입력을 넣으니, 역시 정확히 일치하는 답변이 출력되었습니다.
<br>
<br>
<br>

In [39]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('데이터 강의 너무 좋아요')
input_seq


array([[302, 423, 308,  79,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [40]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


1/1 [==============================] - 0s 42ms/step


'마음 이 아픈가요 '

최고의 강의입니다
<br>
<br>
<br>

In [41]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('4박5일 욜로가려고요')
input_seq

array([[  3,  19,   3,   3, 327,   3,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [42]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence

1/1 [==============================] - 0s 40ms/step


'혼자 도 좋아요 '

하지만 데이터셋에 없던 '4박5일, 욜로'로 입력을 수정하니, 전혀 다른 문장이 출력되었습니다.<br>
이는 우리가 데이터의 일부인 100개 문장만 학습했기 때문입니다.<br>
데이터의 개수를 늘려서 훈련할수록 일반화 능력이 더욱 높아집니다.